In [1]:
from kfp import dsl

@dsl.component
def say_hello(name: str) -> str:
    hello_text = f'Hello, {name}!'
    print(hello_text)
    return hello_text

@dsl.pipeline
def hello_pipeline(recipient: str) -> str:
    hello_task = say_hello(name=recipient)
    return hello_task.output

/home/ubuntu/Desktop/Demetra/venv/lib/python3.12/site-packages/kfp/dsl/component_decorator.py:119: FutureWarning: The default base_image used by the @dsl.component decorator will switch from 'python:3.8' to 'python:3.9' on Oct 1, 2024. To ensure your existing components work with versions of the KFP SDK released after that date, you should provide an explicit base_image argument and ensure your component works as intended on Python 3.9.
  return component_factory.create_component_from_func(


In [2]:
from kfp import compiler

compiler.Compiler().compile(hello_pipeline, 'pipeline.yaml')

In [3]:
from kfp import dsl

@dsl.component
def add(a: float, b: float) -> float:
  '''Calculates sum of two arguments'''
  return a + b



In [29]:
@component(
    packages_to_install=["pandas", "numpy"],
    base_image="python:3.8"  # Use an appropriate base image
)
def generate_data(output_data_path: str):
    import pandas as pd
    import numpy as np
    
    data = {
        'feature1': np.random.rand(100),
        'feature2': np.random.rand(100),
        'label': np.random.randint(0, 2, 100)
    }
    
    df = pd.DataFrame(data)
    df.to_csv(output_data_path, index=False)

@component(
    packages_to_install=["pandas", "numpy"],
    base_image="python:3.8"  # Use an appropriate base image
)
def preprocess_data(input_data_path: str, output_data_path: str):
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    
    df = pd.read_csv(input_data_path)
    scaler = StandardScaler()
    df[['feature1', 'feature2']] = scaler.fit_transform(df[['feature1', 'feature2']])
    
    df.to_csv(output_data_path, index=False)


@component(
    packages_to_install=["pandas", "numpy"],
    base_image="python:3.8"  # Use an appropriate base image
)
def train_model(input_data_path: str):
    import pandas as pd
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score
    import joblib
    import os
    
    df = pd.read_csv(input_data_path)
    X = df[['feature1', 'feature2']]
    y = df['label']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f'Model accuracy: {accuracy}')
    
    model_path = '/tmp/model.joblib'
    joblib.dump(model, model_path)
    
    # Optionally, save model path to be returned
    return model_path



In [31]:
@dsl.pipeline(
    name='Simple Pipeline',
    description='A simple pipeline that generates, preprocesses data, and trains a model.'
)
def simple_pipeline():
    data_output = generate_data()  # Generates data and outputs the path
    preprocessed_output = preprocess_data_op(input_data_path=data_output.output, output_data_path='/tmp/preprocessed_data.csv')  # Specify output path
    train_model_op(input_data_path=preprocessed_output.output)  # Train the model with the preprocessed data


TypeError: generate-data() missing 1 required argument: output_data_path.

In [18]:
if __name__ == '__main__':
    kfp.compiler.Compiler().compile(simple_pipeline, 'simple_pipeline.yaml')
